In [2]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import networkx as nx
"""
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
"""
import warnings
warnings.filterwarnings('ignore')

# Reset display options to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')


In [3]:
df=pd.read_csv('CSVs/train.csv',sep="\t")
display(df)

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,33.0,33.0,NaN,1,0,0,1,NaN,NaN
1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,15.0,15.0,NaN,1,0,0,1,NaN,NaN
2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,78.0,NaN,78.0,0,2,0,1,Define Variable,1
3,9941,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:25:51.0,2005-09-09 12:27:24.0,2005-09-09 12:27:24.0,104.0,NaN,104.0,0,4,9,1,"Using small numbers~~Write expression, positiv...",1~~1~~1
4,9942,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R4C1,2005-09-09 12:27:24.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,6.0,6.0,NaN,1,0,0,1,Entering a given,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232739,1078856,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R5C1,2006-04-12 10:17:08.0,2006-04-12 10:17:17.0,2006-04-12 10:18:19.0,2006-04-12 10:18:19.0,71.0,NaN,71.0,0,2,5,1,"Entering a given~~Convert unit, mixed",21~~1
232740,1078857,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R1C2,2006-04-12 10:18:19.0,2006-04-12 10:18:40.0,2006-04-12 10:18:40.0,2006-04-12 10:18:40.0,21.0,21.0,NaN,1,0,0,1,NaN,NaN
232741,1078858,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R2C2,2006-04-12 10:18:40.0,2006-04-12 10:18:44.0,2006-04-12 10:18:44.0,2006-04-12 10:18:44.0,4.0,4.0,NaN,1,0,0,1,Identifying units,7
232742,1078859,JP20cryBg8,"Unit CTA1_24, Section CTA1_24-1",NOV05,1,R3C2,2006-04-12 10:18:44.0,2006-04-12 10:18:58.0,2006-04-12 10:20:55.0,2006-04-12 10:20:55.0,131.0,NaN,131.0,0,1,0,1,"Using small numbers~~Write expression, positiv...",8~~5~~11


In [4]:
df[['Unit', 'Section']] = df['Problem Hierarchy'].str.split(', ', expand=True)
# Remove the words "Unit " and "Section " from the new columns
df['Unit'] = df['Unit'].str.replace('Unit ', '', regex=False)
df['Section'] = df['Section'].str.replace('Section ', '', regex=False)
df['KC(Default)'] = df['KC(Default)'].fillna('nan')
df['Opportunity(Default)'] = df['Opportunity(Default)'].fillna('nan')


In [5]:
def clean_skill(skill):
    # Patrón para buscar texto entre los primeros dos puntos y después de los primeros puntos y comas
    if str(skill)[0]=='[':
        if "SkillRule: done no solutions" in skill:
            return "done no solutions"
        else:
            skill=skill[skill.index(' ')+1:]
            skill=skill[:skill.index(';')]
            return skill
    
    else:
        return skill

In [6]:
df['KC(Default)']=df['KC(Default)'].apply(clean_skill)

#Reemplazar la columna problem Hierarchy por unit y section

dfa=df.drop(columns=['Problem Hierarchy'])
df=dfa.reindex(columns=['Row', 'Anon Student Id', 'Unit',
       'Section', 'Problem Name',
       'Problem View', 'Step Name', 'Step Start Time',
       'First Transaction Time', 'Correct Transaction Time', 'Step End Time',
       'Step Duration (sec)', 'Correct Step Duration (sec)',
       'Error Step Duration (sec)', 'Correct First Attempt', 'Incorrects',
       'Hints', 'Corrects', 'KC(Default)', 'Opportunity(Default)'])

In [7]:
df=df.drop(columns=['First Transaction Time','Correct Transaction Time','Correct Step Duration (sec)'])

In [ ]:

# Contar cuántos alumnos únicos han hecho cada problema
alumnos_por_problema = df.groupby('Problem Name')['Anon Student Id'].nunique().reset_index()
alumnos_por_problema.columns = ['Problem Name', 'Num Students']

# Identificar los problemas que tienen al menos 10 alumnos
problemas_validos = alumnos_por_problema[alumnos_por_problema['Num Students'] => 10]['Problem Name']

# Filtrar el DataFrame original para mantener solo los problemas válidos
df_alumnos = df[df['Problem Name'].isin(problemas_validos)]
df_alumnos['Problem Name'].unique()


In [9]:
def clasificar_ecuacion(ecuacion):
    # Remover espacios en blanco para facilitar el análisis
    ecuacion = ecuacion.replace(" ", "")
    
    # Detectar términos al cuadrado con variables, raíces cuadradas y coeficientes superiores
    cuadratica = re.search(r'([a-zA-Z]+)\^2', ecuacion) # Detecta variables al cuadrado (solo letras)
    raiz_cuadrada = re.search(r'sqrt', ecuacion)
    coeficiente_superior = re.search(r'([a-zA-Z]+)\^[3-9]', ecuacion)
        
    # Clasificar la ecuación
    if raiz_cuadrada:
        return "sqrt"
    elif coeficiente_superior:
        return "coef 3"
    elif cuadratica:
        return "coef 2"
    else:
        return "coef 1"




In [10]:

filtered_df = df_alumnos[df_alumnos['Step Name'].str.contains('=', na=False)]

# Aplicar la clasificación y agregarla a una nueva columna
df_alumnos.loc[filtered_df.index, 'Step Name'] = filtered_df['Step Name'].apply(clasificar_ecuacion)

print(len(list(df['Step Name'].unique())))

60709


In [11]:
def transform_specific_opportunities(df):
    # Función para tomar solo la primera oportunidad
    def get_first_opportunity(opportunity):
        return str(opportunity).split('~~')[0]

    # Aplicar la transformación solo donde se cumple la condición
    for index, row in df.iterrows():
        kc_list = str(row['KC(Default)']).split('~~')
        opp_list = str(row['Opportunity(Default)']).split('~~')

        # Verificar la condición específica: 1 KC y 2 Opportunities
        if len(kc_list) != len(opp_list):
            df.at[index, 'Opportunity(Default)'] = get_first_opportunity(row['Opportunity(Default)'])

    return df

# Transformar el DataFrame y mostrar el resultado
transformed_df = transform_specific_opportunities(df_alumnos)


KeyboardInterrupt: 

In [ ]:
transformed_df

,Row,Anon Student Id,Unit,Section,Problem Name,Problem View,Step Name,Step Start Time,Step End Time,Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,9938,52vEY7f17k,CTA1_13,CTA1_13-1,PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,33.0,NaN,1,0,0,1,nan,nan
1,9939,52vEY7f17k,CTA1_13,CTA1_13-1,PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,15.0,NaN,1,0,0,1,nan,nan
2,9940,52vEY7f17k,CTA1_13,CTA1_13-1,PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:40.0,78.0,78.0,0,2,0,1,Define Variable,1
3,9941,52vEY7f17k,CTA1_13,CTA1_13-1,PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:27:24.0,104.0,104.0,0,4,9,1,"Using small numbers~~Write expression, positiv...",1~~1~~1
4,9942,52vEY7f17k,CTA1_13,CTA1_13-1,PROP04,1,R4C1,2005-09-09 12:27:24.0,2005-09-09 12:27:30.0,6.0,NaN,1,0,0,1,Entering a given,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232739,1078856,JP20cryBg8,CTA1_24,CTA1_24-1,NOV05,1,R5C1,2006-04-12 10:17:08.0,2006-04-12 10:18:19.0,71.0,71.0,0,2,5,1,"Entering a given~~Convert unit, mixed",21~~1
232740,1078857,JP20cryBg8,CTA1_24,CTA1_24-1,NOV05,1,R1C2,2006-04-12 10:18:19.0,2006-04-12 10:18:40.0,21.0,NaN,1,0,0,1,nan,nan
232741,1078858,JP20cryBg8,CTA1_24,CTA1_24-1,NOV05,1,R2C2,2006-04-12 10:18:40.0,2006-04-12 10:18:44.0,4.0,NaN,1,0,0,1,Identifying units,7
232742,1078859,JP20cryBg8,CTA1_24,CTA1_24-1,NOV05,1,R3C2,2006-04-12 10:18:44.0,2006-04-12 10:20:55.0,131.0,131.0,0,1,0,1,"Using small numbers~~Write expression, positiv...",8~~5~~11


In [ ]:
transformed_df.to_csv("CSVs/CLEAN_DATASET.csv",index=False)

In [ ]:
"""def compare_kc_opportunity(df):
    # Inicializar listas para almacenar las filas que cumplen la condición
    filtered_rows = []

    for index, row in df.iterrows():
        kc_list = str(row['KC(Default)']).split('~~')
        opp_list = str(row['Opportunity(Default)']).split('~~')

        # Comparar la longitud de las listas de KC y Opportunity
        if len(kc_list) != len(opp_list):
            filtered_rows.append(row)

    # Crear un nuevo DataFrame con las filas filtradas
    filtered_df = pd.DataFrame(filtered_rows)

    return filtered_df

# Llamar a la función y mostrar el DataFrame resultante
filtered_df_borrar = compare_kc_opportunity(transformed_df)"""


"def compare_kc_opportunity(df):\n    # Inicializar listas para almacenar las filas que cumplen la condición\n    filtered_rows = []\n\n    for index, row in df.iterrows():\n        kc_list = str(row['KC(Default)']).split('~~')\n        opp_list = str(row['Opportunity(Default)']).split('~~')\n\n        # Comparar la longitud de las listas de KC y Opportunity\n        if len(kc_list) != len(opp_list):\n            filtered_rows.append(row)\n\n    # Crear un nuevo DataFrame con las filas filtradas\n    filtered_df = pd.DataFrame(filtered_rows)\n\n    return filtered_df\n\n# Llamar a la función y mostrar el DataFrame resultante\nfiltered_df_borrar = compare_kc_opportunity(transformed_df)"

In [ ]:

dict_analisis_outliers = {}
Units = list(transformed_df['Unit'].unique())
for unit in Units:
    print(unit)
    df_unit = transformed_df[transformed_df['Unit'] == str(unit)]
    Sections = list(df_unit['Section'].unique())
    dict_analisis_outliers[unit] = {}
    
    for section in Sections:
        df_section = df_unit[df_unit['Section'] == section]
        problems = list(df_section['Problem Name'].unique())
        dict_analisis_outliers[unit][section] = {}
        
        for problem in problems:
            df_problem = df_section[df_section['Problem Name'] == problem]
            alumnos = list(df_problem['Anon Student Id'].unique())
            dict_analisis_outliers[unit][section][problem] = {}
            
            steps = list(df_problem['Step Name'].unique())
            for step in steps:
                df_step = df_problem[df_problem['Step Name'] == step]
                kcs = list(df_step['KC(Default)'].unique())
                for kc in kcs:
                    step_kc_data = df_step[df_step['KC(Default)'] == kc]
                    tuples_list = []
                    for _, row in step_kc_data.iterrows():
                        problem_view = row['Problem View']
                        failures = row['Incorrects'] / problem_view
                        hints = row['Hints'] / problem_view
                        corrects = row['Corrects'] / problem_view
                        student_id = row['Anon Student Id']
                        tuples_list.append((failures, hints, corrects, student_id))
                    dict_analisis_outliers[unit][section][problem][str((step, kc))] = tuples_list



CTA1_13
CTA1_06
ES_03
CTA1_08
ES_04
CTA1_10
ES_07
ES_02
CTA1_12
ES_05
CTA1_14
LINEAR-INEQUALITY-GRAPHING
CTA1_15
QUAD-ADD-AREA-ALG1
QUADRATICS-FACTORING_ES
QUADRATICS-SOLVING_ES
QUAD-VERTICAL-MOTION
CTA1_01
EXPT-PRODUCT-SIMP-A_ES
EXPT-QUOTIENT-SIMP-A_ES
CTA1_20
CTA1_24
CTA1_02
ES_01
CTA1_04


In [ ]:
import json
with open('nested_data_clust.json', 'w') as json_file:
    json.dump(dict_analisis_outliers, json_file, indent=4)


In [ ]:
import json
json_file_path = 'nested_data_clust.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

In [ ]:

def detect_outliers(data):
    mean = np.mean(data)
    threshold = 5 * mean 
    return (data > threshold) & (data>10)
# Filtrar outliers en dict_analisis_outliers y recopilar outliers detectados
filtered_dict = {}
outliers_detected = []

for unit, sections in data.items():
    filtered_dict[unit] = {}
    for section, problems in sections.items():
        filtered_dict[unit][section] = {}
        for problem, step_kcs in problems.items():
            filtered_dict[unit][section][problem] = {}
            for keyz, tuples_list in step_kcs.items():
                # Extraer failures y hints
                failures = [t[0] for t in tuples_list]
                hints = [t[1] for t in tuples_list]
                
                # Detectar outliers
                outliers_failures = detect_outliers(np.array(failures))
                outliers_hints = detect_outliers(np.array(hints))
                
                # Filtrar los datos que no son outliers y recopilar los outliers
                filtered_tuples_list = []
                for i, t in enumerate(tuples_list):
                    if outliers_failures[i] or outliers_hints[i]:
                        outliers_detected.append({
                            'unit': unit,
                            'section': section,
                            'problem': problem,
                            'step': tuple(keyz)[0],
                            'student': t[3],
                            'failures': t[0],
                            'hints': t[1],
                            'corrects': t[2]
                        })
                    else:
                        filtered_tuples_list.append(t)
                
                filtered_dict[unit][section][problem][keyz] = filtered_tuples_list

# Mostrar outliers detectados
outliers_df = pd.DataFrame(outliers_detected)
#print("Outliers detectados y eliminados:")
#print(outliers_df.to_string(index=False))

# Imprimir en un formato más estilizado
print("\nOutliers detectados y eliminados (formato estilizado):")
for outlier in outliers_detected:
    print(f"Unidad: {outlier['unit']}, Sección: {outlier['section']}, Problema: {outlier['problem']}, Paso: {outlier['step']}, Alumno: {outlier['student']}, "
          f"Hints: {outlier['hints']}, Errores: {outlier['failures']}, Aciertos: {outlier['corrects']}")



Outliers detectados y eliminados (formato estilizado):
Unidad: CTA1_13, Sección: CTA1_13-1, Problema: PROP04, Paso: (, Alumno: 242fhbKRf9, Hints: 13.0, Errores: 1.0, Aciertos: 1.0
Unidad: CTA1_13, Sección: CTA1_13-1, Problema: PROP07, Paso: (, Alumno: 6W08a98ZQV, Hints: 4.0, Errores: 12.0, Aciertos: 1.0
Unidad: CTA1_06, Sección: CTA1_06-2, Problema: BH1T20, Paso: (, Alumno: 242fhbKRf9, Hints: 2.0, Errores: 18.0, Aciertos: 0.0
Unidad: CTA1_06, Sección: CTA1_06-2, Problema: FEB11, Paso: (, Alumno: E38It7fjg0, Hints: 13.0, Errores: 18.0, Aciertos: 1.0
Unidad: CTA1_06, Sección: CTA1_06-2, Problema: L5FB16, Paso: (, Alumno: ey9rvMnU57, Hints: 12.0, Errores: 3.0, Aciertos: 0.0
Unidad: CTA1_06, Sección: CTA1_06-2, Problema: L5FB16, Paso: (, Alumno: 5Q69X4zFkC, Hints: 4.0, Errores: 12.0, Aciertos: 0.0
Unidad: CTA1_06, Sección: CTA1_06-2, Problema: L5FB16, Paso: (, Alumno: q1Yar3mIhE, Hints: 8.0, Errores: 17.0, Aciertos: 1.0
Unidad: CTA1_06, Sección: CTA1_06-2, Problema: L5FB16, Paso: (, Alumn

In [ ]:
with open('filtered_nested_data_clust.json', 'w') as json_file:
    json.dump(filtered_dict, json_file, indent=4)
